<a href="https://colab.research.google.com/github/worachot-n/Text_summarization_T5/blob/main/1_6_Text_Summarization_Evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Set Location Folder

In [2]:
import os
os.chdir('/content/drive/MyDrive/Kaggle/outputs')

# Install Library

In [3]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]
!pip install datasets nltk
!pip install bert-score
!pip install rouge

     |████████████████████████████████| 1.2 MB 8.7 MB/s 
     |████████████████████████████████| 4.0 MB 7.4 MB/s 
     |████████████████████████████████| 895 kB 45.8 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
     |████████████████████████████████| 6.6 MB 27.1 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 229 kB 7.1 MB/s 
     |████████████████████████████████| 51 kB 6.2 MB/s 
     |████████████████████████████████| 325 kB 7.7 MB/s 
     |████████████████████████████████| 136 kB 33.4 MB/s 
     |████████████████████████████████| 212 kB 41.1 MB/s 
     |████████████████████████████████| 1.1 MB 38.9 MB/s 
     |████████████████████████████████| 127 kB 55.1 MB/s 
     |████████████████████████████████| 144 kB 44.0 MB/s 
     |████████████████████████████████| 94 kB 2

# Import Library

In [4]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

from datasets import load_dataset, load_metric

from bert_score import BERTScorer
from rouge import Rouge

import difflib
import nltk
import re

%matplotlib inline

# Import Dataset

In [13]:
dataset_evaluate = pd.read_csv('/content/drive/MyDrive/Kaggle/outputs/predictions_business_business.csv')

In [6]:
dataset_evaluate.head()

,Full Text,Generated Text,Actual Text,Finetune,Category
0,Peugeot deal boosts Mitsubishi Struggling Japa...,Japanese car maker Mitsubishi Motors has struc...,Struggling Japanese car maker Mitsubishi Motor...,business,business
1,Japan bank shares up on link talk Shares of Su...,"SMFG shares climbed by 2.7% to 717,000, and Da...","Shares of Sumitomo Mitsui Financial (SMFG), an...",business,business
2,Ebbers 'aware' of WorldCom fraud Former WorldC...,"Mr Ebbers has denied all the charges, saying h...",Mr Sullivan is at the centre of the government...,business,business
3,Barclays shares up on merger talk Shares in UK...,Shares in UK banking group Barclays have risen...,Shares in UK banking group Barclays have risen...,business,business
4,French boss to leave EADS The French co-head o...,The company should now be able put this disput...,The company should now be able put this disput...,business,business


In [8]:
rouge = Rouge()

In [16]:
rouge.get_scores(dataset_evaluate['Generated Text'].to_list(), dataset_evaluate['Actual Text'].to_list())

[{'rouge-1': {'f': 0.638036804847755,
   'p': 0.5909090909090909,
   'r': 0.6933333333333334},
  'rouge-2': {'f': 0.5675675625679735, 'p': 0.5727272727272728, 'r': 0.5625},
  'rouge-l': {'f': 0.638036804847755,
   'p': 0.5909090909090909,
   'r': 0.6933333333333334}},
 {'rouge-1': {'f': 0.608108103130935,
   'p': 0.6521739130434783,
   'r': 0.569620253164557},
  'rouge-2': {'f': 0.43820224221373566,
   'p': 0.46987951807228917,
   'r': 0.4105263157894737},
  'rouge-l': {'f': 0.5945945896174215,
   'p': 0.6376811594202898,
   'r': 0.5569620253164557}},
 {'rouge-1': {'f': 0.5890410908988554,
   'p': 0.5657894736842105,
   'r': 0.6142857142857143},
  'rouge-2': {'f': 0.5099999950080001, 'p': 0.49038461538461536, 'r': 0.53125},
  'rouge-l': {'f': 0.5753424607618691, 'p': 0.5526315789473685, 'r': 0.6}},
 {'rouge-1': {'f': 0.7151515101825529,
   'p': 0.7763157894736842,
   'r': 0.6629213483146067},
  'rouge-2': {'f': 0.6912442346552274,
   'p': 0.7425742574257426,
   'r': 0.646551724137931},

In [ ]:
dataset_evaluate['rouge-1'] = dataset_evaluate.apply(lambda row: rouge.get_scores(row['Generated Text'].to_list()), axis=1)

In [17]:
refs = dataset_evaluate['Actual Text'].to_list()

hyps = dataset_evaluate['Generated Text'].to_list()

scorer = BERTScorer(lang="en", rescale_with_baseline=True)
P, R, F1 = scorer.score(hyps, refs)

print(F1)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([0.5809, 0.5288, 0.5220, 0.6333, 0.7921, 0.5104, 0.4817, 0.7010, 0.6112,
        0.7123, 0.6714, 0.7084, 0.5803, 0.5525, 0.5109, 0.4929, 0.4434, 0.8202,
        0.4190, 0.4625, 0.7222, 0.7265, 0.3815, 0.6636, 0.3982, 0.7703, 0.6241,
        0.6012, 0.4260, 0.4419, 0.3464, 0.6127, 0.6050, 0.7192, 0.5565, 0.6646,
        0.6353, 0.7976, 0.4848, 0.5654, 0.5326, 0.3286, 0.3960, 0.3468, 0.4432,
        0.6468, 0.4619])


In [20]:
'''
Find the matching substrings in 2 strings.
:parameter
    :param a: string - raw text
    :param b: string - raw text
:return
    2 lists used in to display matches
'''
def utils_split_sentences(a, b):
    ## find clean matches
    match = difflib.SequenceMatcher(isjunk=None, a=a, b=b, autojunk=True)
    lst_match = [block for block in match.get_matching_blocks() if block.size > 20]
    
    ## difflib didn't find any match
    if len(lst_match) == 0:
        lst_a, lst_b = nltk.sent_tokenize(a), nltk.sent_tokenize(b)
    
    ## work with matches
    else:
        first_m, last_m = lst_match[0], lst_match[-1]

        ### a
        string = a[0 : first_m.a]
        lst_a = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = a[m.a : m.a+m.size]
            lst_a.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = a[m.a+m.size : next_m.a]
                lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = a[last_m.a+last_m.size :]
        lst_a = lst_a + [t for t in nltk.sent_tokenize(string)]

        ### b
        string = b[0 : first_m.b]
        lst_b = [t for t in nltk.sent_tokenize(string)]
        for n in range(len(lst_match)):
            m = lst_match[n]
            string = b[m.b : m.b+m.size]
            lst_b.append(string)
            if n+1 < len(lst_match):
                next_m = lst_match[n+1]
                string = b[m.b+m.size : next_m.b]
                lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
            else:
                break
        string = b[last_m.b+last_m.size :]
        lst_b = lst_b + [t for t in nltk.sent_tokenize(string)]
    
    return lst_a, lst_b


'''
Highlights the matched strings in text.
:parameter
    :param a: string - raw text
    :param b: string - raw text
    :param both: bool - search a in b and, if True, viceversa
    :param sentences: bool - if False matches single words
:return
    text html, it can be visualized on notebook with display(HTML(text))
'''
def display_string_matching(a, b, both=True, sentences=True, titles=[]):
    if sentences is True:
        lst_a, lst_b = utils_split_sentences(a, b)
    else:
        lst_a, lst_b = a.split(), b.split()       
    
    ## highlight a
    first_text = []
    for i in lst_a:
        if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_b]:
            first_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
        else:
            first_text.append(i)
    first_text = ' '.join(first_text)
    
    ## highlight b
    second_text = []
    if both is True:
        for i in lst_b:
            if re.sub(r'[^\w\s]', '', i.lower()) in [re.sub(r'[^\w\s]', '', z.lower()) for z in lst_a]:
                second_text.append('<span style="background-color:rgba(255,215,0,0.3);">' + i + '</span>')
            else:
                second_text.append(i)
    else:
        second_text.append(b) 
    second_text = ' '.join(second_text)
    
    ## concatenate
    if len(titles) > 0:
        first_text = "<strong>"+titles[0]+"</strong><br>"+first_text
    if len(titles) > 1:
        second_text = "<strong>"+titles[1]+"</strong><br>"+second_text
    else:
        second_text = "---"*65+"<br><br>"+second_text
    final_text = first_text +'<br><br>'+ second_text
    return final_text

In [30]:
match1 = display_string_matching(dataset_evaluate['Full Text'].to_list()[0], dataset_evaluate['Actual Text'].to_list()[0], both=True, sentences=False, titles=["Full Text", "Actual Text"])
from IPython.core.display import display, HTML
display(HTML(match1))

In [31]:
match2 = display_string_matching(dataset_evaluate['Full Text'].to_list()[0], dataset_evaluate['Generated Text'].to_list()[0], both=True, sentences=False, titles=["Full Text", "Generated Text"])
from IPython.core.display import display, HTML
display(HTML(match2))

In [32]:
match3 = display_string_matching(dataset_evaluate['Actual Text'].to_list()[0], dataset_evaluate['Generated Text'].to_list()[0], both=True, sentences=False, titles=["Actual Text", "Generated Text"])
from IPython.core.display import display, HTML
display(HTML(match3))